# Introduction to metaprogramming

Normal programs manipulate data. Metaprograms **manipulate code**. (In many languages this is impossible.)
Of course, the ultimate goal is that the programs that we manipulate will later operate on data.

## Expressions

We use the special syntax `:( ... )` to represent a piece of Julia code:

In [3]:
x + 3  # tries to evaluate the variable x (which doesn't exist)

LoadError: UndefVarError: x not defined

In [4]:
ex = :(x + 3)   # "the unevaluated piece of code x + 3"

:(x + 3)

We can look at the structure of the object `ex` with the `dump` function:

In [5]:
ex

:(x + 3)

In [6]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol x
    3: Int64 3
  typ: Any


In [ ]:
typeof(ex)

We see that `ex` is of type `Expr`, meaning a Julia expression object - a piece of Julia code, stored as an object inside Julia itself!

The details about the object are stored (as usual) in its fields. The kind of expression object is stored as the `head` field:

In [7]:
ex.head

:call

In this case, we see that it represents a function call. 

The arguments of the function call are stored in the `ex.args` field:

In [8]:
ex.args

3-element Array{Any,1}:
  :+
  :x
 3  

In [9]:
ex.args[2]

:x

In [10]:
ex.args[2] = :y

:y

In [11]:
ex

:(y + 3)

We can now *manipulate* and *modify* the object by *changing* these fields:

In [ ]:
ex.args[2] = :z
ex.args[3] = :4

In [ ]:
ex

We see that the expression has changed. We can even change it from an addition to something else:

In [ ]:
ex.args[1] = :*

In [ ]:
ex

This is the basis of metaprogramming, since it gives us the ability to take one piece of code, and modify it to produce another piece of code.

The foundation of an expression are `Symbol`s, written with initial colons (`:`):


In [ ]:
typeof(:z)

A more compact form to display an expression is as follows:

In [ ]:
Meta.show_sexpr(ex)

## More complicated expressions

What about more complicated expressions?

In [ ]:
ex2 = :(x + 3y)

In [ ]:
Meta.show_sexpr(ex2)

In [ ]:
dump(ex2)

In [ ]:
ex2.args[3]

In [ ]:
typeof(ex2.args[3])

We see that expressions are recursive, in the sense that the subpiece `:(3y)` is itself another `Expr` object.

Julia provides a mechanism to create an `Expr` from a `String`, via the `parse` function:

In [ ]:
parse("x + 3y")

## Manipulating expressions

How can we do something more complicated? For example, suppose we start with the expression

In [ ]:
ex = :(x + 3x^2)

and we would like to change *all* `x`s to `(x+1)`s? We will need to dig down into the structure.
A first attempt is the following function. Note that this *modifies* its argument, so has a `!` ("bang") in the function name.

In [ ]:
function traverse!(ex::Expr)
    
    args = ex.args
    
    for i in 1:length(args)
        
        if args[i] == :x
            args[i] = :(x+1)
        end
        
    end
end

In [ ]:
traverse!(ex)

In [ ]:
ex

Oops, that didn't work -- we didn't manage to reach inside the inner `Expr` object. 

In [ ]:
ex = :(x + 3x^2)

In [ ]:
function traverse!(ex::Expr)
    
    args = ex.args
    
    for i in 1:length(args)
        @show i, args[i]
        if args[i] == :x
            args[i] = :(x+1)
        end
    end
end

In [ ]:
traverse!(ex)

The AST ("Abstract Syntax Tree") has a **recursive** structure, so we use a recursive function:

In [ ]:
Meta.show_sexpr(ex)

In [ ]:
function traverse!(ex::Expr)
    
    args = ex.args
    
    for i in 1:length(args)
        @show i, args[i]
        
        if isa(args[i], Expr)
            traverse!(args[i])
            
        elseif args[i] == :x
            args[i] = :(x+1)
        end
    end
end

In [ ]:
traverse!(ex)

In [ ]:
ex

**Exercise**: Make the `traverse!` function more general, to replace `x` by an arbitrary expression that is another argument of the function.

## Evaluation

Now that we have our new object, we want to use it in Julia. We do this using `eval`:


In [ ]:
ex = :(3x^2)

In [ ]:
eval(ex)

In [ ]:
x = 3

In [ ]:
eval(ex)

## Using multiple dispatch with expressions

What if, say, we want to wrap "number literals" like 3, 4.5 with `f(3)` etc.?
We will write a more Julian version, using **multiple dispatch**:

In [ ]:
function wrap_literals(ex::Expr, f::Symbol)
    
    #println("Entering !")
    #@show ex, s, new_expr
    
    args = ex.args
    
    for i in 1:length(args)
        #@show i, args[i]
        args[i] = wrap_literals(args[i], f)
        
    end
    
    return ex
end

wrap_literals(x::Number, f::Symbol) = :($f($x))
wrap_literals(x, f) = x  # fall-back method

In [ ]:
ex = :(3x^2)
wrap_literals(:(3x^2), :f)

## Inserting pieces of code

Wilkinson-type polynomials are polynomials like

$p_5(x) = (x-1) (x-2) (x-3) (x-4) (x-5)$

In Julia we could write

In [5]:
p_5(x) = (x-1) * (x-2) * (x-3) * (x-4) * (x-5)

p_5 (generic function with 1 method)

**Exercise:** We can interpolate pieces of code in other pieces using the `$` operator, and symbols from strings using `Symbol`. Make and evaluate an expression to define $p_n$ for different $n$, and interact with it.

## Generating repetitive code

One of the uses of metaprogramming is **code generation**. 
Suppose that for some reason we want to make a type that behaves like a `Float64`, but with some additional functionality.:

In [4]:
struct MyFloat
    a::Float64
end

We would need to write methods like


In [ ]:
import Base: +, -, *

+(x::MyFloat, y::MyFloat) = x.a + y.a
-(x::MyFloat, y::MyFloat) = x.a - y.a
*(x::MyFloat, y::MyFloat) = x.a * y.a

This will get boring fast. Whenever we have repetition like this, we should get the computer to do it for us. However, here we need to **repeat code**. Let's make a code *template*; it should look like

    op(x::MyFloat, y::MyFloat) = op(x.a, y.a)

We want a code object that looks like that:

In [ ]:
ex = :(op(x::MyFloat, y::MyFloat) = op(x.a, y.a))

We can also write this as 

In [1]:
ex = quote
        op(x::MyFloat, y::MyFloat) = op(x.a, y.a)
     end

quote  # In[1], line 2:
    op(x::MyFloat, y::MyFloat) = begin  # In[1], line 2:
            op(x.a, y.a)
        end
end

However, we want to replace `op` **by its value**

In [2]:
op = :+
ex = quote
        $op(x::MyFloat, y::MyFloat) = $op(x.a, y.a)
     end

quote  # In[2], line 3:
    x::MyFloat + y::MyFloat = begin  # In[2], line 3:
            x.a + y.a
        end
end

And now just loop:

In [5]:
import Base: +,-,*,/

for op in (:+, :-, :*, :/)
    
    ex = quote
        $op(x::MyFloat, y::MyFloat) = $op(x.a, y.a)
    end
    
    eval(ex)
end


In [7]:
x = MyFloat(3)
y = MyFloat(4)

MyFloat(4.0)

In [8]:
x + y

7.0

In [9]:
x - y

-1.0

Julia provides `@eval` to make this shorter:

In [ ]:
for op in (:+, :-, :*, :/)
    
    @eval $op(x::MyFloat, y::MyFloat) = $op(x.a, y.a)
    
end


## Macros

We don't want the user to have to think about Julia syntax objects. We can supply a **macro**.
This can be thought of as a "meta-function": it takes a Julia expression object as argument, and returns a Julia expression object. However, when it is called, it *automatically parses its input into the expression object*.

The simplest example:

In [ ]:
macro simple(ex)
    show(ex)
    return nothing
end   

In [ ]:
@simple 3x

We can now make a macro `wrap` that calls our function:

In [ ]:
macro wrap(f, ex)
    new_ex = wrap_literals(ex, f)
    show(new_ex)
end

In [ ]:
@wrap g 3x+2

We can also call it as `@wrap(g, 3x+2)`:

In [ ]:
@wrap(g, 3x+2)

In fact, macros should **return an expression**. This expression will be **evaluated in the scope in which it is called**:

In [ ]:
macro wrap(f, ex)
    new_ex = wrap_literals(ex, f)
    return new_ex
end

In [ ]:
@wrap(g, 3x+2)

In [ ]:
x = 3
g(x) = 10x

In [ ]:
@wrap g 3x+2

In [ ]:
g(3)*x+g(2)

## Example of macro: @time 

In [12]:
@time sin(10)

  0.000002 seconds (5 allocations: 176 bytes)


-0.5440211108893698

In [13]:
@macroexpand @time sin(10)

quote  # util.jl, line 235:
    local #10#stats = (Base.gc_num)() # util.jl, line 236:
    local #12#elapsedtime = (Base.time_ns)() # util.jl, line 237:
    local #11#val = sin(10) # util.jl, line 238:
    #12#elapsedtime = (Base.time_ns)() - #12#elapsedtime # util.jl, line 239:
    local #13#diff = (Base.GC_Diff)((Base.gc_num)(), #10#stats) # util.jl, line 240:
    (Base.time_print)(#12#elapsedtime, #13#diff.allocd, #13#diff.total_time, (Base.gc_alloc_count)(#13#diff)) # util.jl, line 242:
    #11#val
end

In [14]:
using IntervalConstraintProgramming

INFO: Recompiling stale cache file /Users/dpsanders/.julia/lib/v0.6/Polynomials.ji for module Polynomials.
INFO: Recompiling stale cache file /Users/dpsanders/.julia/lib/v0.6/IntervalConstraintProgramming.ji for module IntervalConstraintProgramming.

Use "abstract type Separator end" instead.

Use "SubPaving{N,T} = Vector{IntervalBox{N,T}}" instead.


In [15]:
C = @constraint x^2 + y^2 <= 1

Separator:
  - variables: x, y
  - expression: x ^ 2 + y ^ 2 ∈ [-∞, 1]

In [19]:
using MacroTools

In [20]:
MacroTools.striplines(C.contractor.forward.code)

quote 
    t->begin 
            begin 
                (x, y) = t
                begin 
                    _a = x ^ 2
                    _b = y ^ 2
                    _c = _a + _b
                end
                return (_c, (_a, _b))
            end
        end
end

In [21]:
MacroTools.striplines(C.contractor.backward.code)

quote 
    (t1, t2, t3)->begin 
            begin 
                (x, y) = t1
                (_c,) = t2
                (_a, _b) = t3
                begin 
                    (_c, _a, _b) = plus_rev(_c, _a, _b)
                    (_b, y, _) = power_rev(_b, y, 2)
                    (_a, x, _) = power_rev(_a, x, 2)
                end
                return (x, y)
            end
        end
end

In [18]:
@macroexpand @constraint x^2 + y^2 <= 1

quote  # /Users/dpsanders/.julia/v0.6/IntervalConstraintProgramming/src/separator.jl, line 125:
    _C1 = (IntervalConstraintProgramming.Contractor)(Symbol[:x, :y], Symbol[:_f], (IntervalConstraintProgramming.GeneratedFunction)(begin  # /Users/dpsanders/.julia/v0.6/IntervalConstraintProgramming/src/code_generation.jl, line 193:
                    #40#t->begin  # /Users/dpsanders/.julia/v0.6/IntervalConstraintProgramming/src/code_generation.jl, line 193:
                            begin  # /Users/dpsanders/.julia/v0.6/IntervalConstraintProgramming/src/code_generation.jl, line 194:
                                (#35#x, #36#y) = #40#t # /Users/dpsanders/.julia/v0.6/IntervalConstraintProgramming/src/code_generation.jl, line 195:
                                begin 
                                    #37#_d = #35#x ^ 2
                                    #38#_e = #36#y ^ 2
                                    #39#_f = #37#_d + #38#_e
                                end # /Users/dpsand